## High dynamic range and environment maps

This example shows how to read a HDR image and setup it as an environment map in the scene background. Files prepared in
the notebook 10_1 are used.

In [1]:
import numpy as np
from plotoptix import TkOptiX
from plotoptix.utils import make_color_2d

Setup ray tracing parameters:

In [2]:
rt = TkOptiX()
rt.set_param(
    min_accumulation_step=2,
    max_accumulation_frames=30
)

exposure = 1; gamma = 2.2
rt.set_float("tonemap_exposure", exposure)
rt.set_float("tonemap_gamma", gamma)
rt.set_float("denoiser_blend", 0.2)
rt.add_postproc("Denoiser")

rt.set_ambient([0.4, 0.6, 0.8])
rt.set_background(0)

Camera:

In [3]:
rt.setup_camera("cam1", cam_type="DoF",
                eye=[-3.5, 2, 3.5], target=[0, 0, 0], up=[0, 1, 0],
                aperture_radius=0.07, fov=50, focal_scale=0.8)

And some data to display:

In [4]:
xyz = np.mgrid[-1.0:1.0:0.25, -1.0:1.0:0.25, -1.0:1.0:0.25].reshape(3,-1).T

# set u and v - makes regular orientation of blocks
#rt.set_data("blocks", pos=xyz, u=[0.2,0,0], v=[0,0.2,0], c=0.7, geom="Parallelepipeds")

# set u vectors only, blocks are randomly rotated
rt.set_data("blocks", pos=xyz, u=[0.2,0,0], c=0.7, geom="Parallelepipeds")

Open the GUI window:

In [5]:
rt.show()

Set background to the environment map prepared in the previous example. Try 8, 16 and 32 bit/sample color depths. **Note** that the texture has to account for the gamma correction to keep the original colors.

In [6]:
rt.set_background("rt_output_16bps.tif", gamma=gamma)

# background texture loaded explicitly, this is what set_background() does when you pass a file name to it
#rt.load_texture("bg_texture", "rt_output_16bps.tif", gamma=gamma)

rt.set_background_mode("TextureEnvironment")

Use HDR image saved in Radiance file (OpenCV required):

In [7]:
import cv2

img = cv2.imread("rt_output_32bps.hdr", cv2.IMREAD_UNCHANGED) # IMREAD_UNCHANGED required to read the 32bps data
print(img.dtype, img.shape, np.max(img))

float32 (1084, 1924, 3) 1.203125


In [8]:
img = make_color_2d(img, gamma=gamma, channel_order="BGRA") # need to account for gamma correction and convert to BGRA format
print(img.dtype, img.shape, np.max(img))

float32 (1084, 1924, 4) 1.5020474


In [9]:
rt.set_background(img, refresh=True)

# background texture loaded explicitly, this is what set_background() does when you pass an array to it
#rt.set_texture_2d("bg_texture", img, refresh=True)

In [10]:
rt.close()